In [22]:
#一句话预处理行为日志

!ls
!cat ./data/ratings_Beauty.csv | grep -v UserId | sort -t',' -nk4 | awk 'BEGIN{FS=",";OFS="\t"}$3>2{d[$1]=d[$1]$2";"}END{for(i in d) print i"\t"substr(d[i],0,length(d[i])-1)}' | awk '{split($2,a,";");if(length(a)>3)print $2}' | awk 'gsub(";","\t",$0)' > ./data/corpus.dat

data          nlprs02.ipynb


In [13]:
#用gensim计算商品的特征向量

import gensim
from gensim.models.word2vec import *
import multiprocessing

#向量维度
v_size = 8
#窗口大小
v_window = 3
#最小频率
v_min_count = 2
v_workers = 2

corpusFilePath = './data/corpus.dat'

corpusFile = open(corpusFilePath, u'r')
model = Word2Vec(
    LineSentence(corpusFile), 
    size=v_size, 
    window=v_window, 
    min_count=v_min_count, workers=v_workers)

modelPath = './data/model.dat'
model.wv.save_word2vec_format(modelPath, binary=False)

In [ ]:
#用annoy建立索引

import csv
from annoy import AnnoyIndex
 
#向量的维度
f = 8
#新建索引的实例
t = AnnoyIndex(f)

#商品序号
NR = 0
#商品序号 -> 商品ID
nr2name = {}
with open('./data/model.dat') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=' ')
    for line in csv_reader:

        NR += 1

        if len(line) != 9: continue

        try:
            trigger = line[0]
            line = line[1:]
            v = [float(dim) for dim in line]
            t.add_item(NR, v)
            nr2name.setdefault(NR, trigger)

        except Exception as e:
            print("error")  

#用20棵树构造索引
t.build(20)
t.save('./data/ann_index')

In [ ]:
#读取模型文件
u = AnnoyIndex(f)
u.load('./data/ann_index')

#展示最近邻
for NR,name in nr2name.items():
    
    if NR > 5: break
        
    print(name + "最相近的item: ")
    
    r = u.get_nns_by_item(NR, 5, include_distances=False)
    for item in r:
        print("\t" + nr2name.get(item))
        
#作业一：
#从这里下载Metadata：http://jmcauley.ucsd.edu/data/amazon/
#来验证通过向量找到的最近邻是否符合预期

In [17]:
#hnsw的参考文献：
#Malkov Y A, Yashunin D A. Efficient and robust approximate nearest neighbor search using hierarchical navigable small world graphs[J]. IEEE transactions on pattern analysis and machine intelligence, 2018.

In [18]:
#以下是DSSM模型相关

In [23]:
import numpy as np
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.layers import Activation, Input, Embedding, Flatten, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, GlobalAveragePooling1D
from tensorflow.keras.layers import dot
from tensorflow.keras import backend as K
from tensorflow.keras.layers import concatenate

from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras

from sklearn.preprocessing import normalize

2.0.0


In [25]:
#DSSM

#模型结构
def dssm(item_voc_size, item_embeddings, seq_len=5, dim=8, J=4):
    
    #模型输入
    user_input = Input(shape=(seq_len,), name='user_input')
    pos_item_input = Input(shape=(1,), name='item_input')
    neg_item_inputs = [Input(shape=(1,)) for _ in range(J)]

    #user塔
    user_embedding = Embedding(item_voc_size, dim, weights=[item_embeddings], input_length=seq_len, trainable=False)(user_input)
    user_avg_pooling = GlobalAveragePooling1D()(user_embedding)
    user_fc = Dense(8, activation='relu', name='ufc')(user_avg_pooling)

    #item塔
    pos_item_embedding = Embedding(item_voc_size, dim, weights=[item_embeddings], trainable=False)(pos_item_input)
    neg_item_embeddings = [Embedding(item_voc_size, dim, weights=[item_embeddings], trainable=False)(neg_item_input) for neg_item_input in neg_item_inputs]

    pos_item_flatten = Flatten()(pos_item_embedding)
    neg_item_flattens = [Flatten()(neg_item_embedding) for neg_item_embedding in neg_item_embeddings]

    item_fc = Dense(8, activation='relu', name='ifc')

    pos_item_fc = item_fc(pos_item_flatten)
    neg_item_fcs = [item_fc(neg_item_flatten) for neg_item_flatten in neg_item_flattens]

    user_item_pos_output = dot([user_fc, pos_item_fc], axes=1, normalize=True)
    user_item_neg_outputs = [dot([user_fc, neg_item_fc], axes=1, normalize=True) for neg_item_fc in neg_item_fcs]

    outputs = concatenate([user_item_pos_output] + user_item_neg_outputs)

    prob = Activation("softmax")(outputs)

    model = Model(inputs=[user_input, pos_item_input] + neg_item_inputs, outputs=prob)
    model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['acc'])
    return model


#作业二：一个基本的dssm模型结构已经搭建完成（如上面代码）。
#现在你可以尝试根据模型所需的输入格式，从原始行为日志生成input data（答案下节课讲）
item_voc_size = 100000
emb_dim = 8
item_embeddings = np.zeros((item_voc_size, emb_dim))
model = dssm(item_voc_size, item_embeddings, seq_len=5)
print(model.summary())

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 5)]          0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_17 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           [(None, 1)]          0                                            
____________________________________________________________________________________________